## IBM Data Science Professional Certificate
### Applied Data Science Capstone

Damien Azzopardi - July 2021

<h2>Table of Contents</h2>

<ol>
    <li><a href="#assignment_1"><b>Assignment 1: Capstone Project Notebook</b></a>
<br>
<br>
    <li><a href="#assignment_2"><b>Assignment 2: Segmenting and Clustering Neighborhoods in Toronto</b></a></li>
        <ul>
            <li><a href="#assignment_2_1">2.1: Scrap Toronto's postal codes</a>
            <li><a href="#assignment_2_2">2.2: Get the latitude and longitude coordinates of each neighborhood</a>
            <li><a href="#assignment_2_3">2.3: Explore and cluster the neighborhoods in Toronto</a>          
        </ul>
<hr>

<h2 id="assignment_1">Assignment 1: Capstone Project Notebook</h2>

In [1]:
# load libraries
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


<h2 id="assignment_2">Assignment 2: Segmenting and Clustering Neighborhoods in Toronto</h2>

In [3]:
# load libraries
import requests
from bs4 import BeautifulSoup
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

<h3 id="assignment_2_1">2.1: Scrap Toronto's postal codes</h3>

The full list of Toronto's postal codes is available in [this Wikipedia page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M).

In [4]:
# scrap Toronto postal codes and create a data frame containing 'PostalCode', 'Borough' and 'Neighborhood'
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = requests.get(url).text
soup = BeautifulSoup(data, 'lxml')

table_contents = []
table = soup.find('table')
for row in table.findAll('td'):
    cell = {}
    # Ignore cells with a borough that is 'Not assigned'
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

df = pd.DataFrame(table_contents)
# clean 'Borough' names
df['Borough'] = df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [5]:
df.shape

(103, 3)

<h3 id="assignment_2_2">2.2: Get the latitude and longitude coordinates of each neighborhood</h3>

In [6]:
# using the Geospatial_Coordinated csv file containing geographical coordinates for each postal code
df_coordinates = pd.read_csv("/Users/damienazzopardi/Documents/GitHub/Coursera_Capstone/Geospatial_Coordinates.csv")
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
df_coordinates.shape

(103, 3)

In [8]:
# merge both dataframes into one
df_toronto = pd.merge(df, df_coordinates, how = 'left', left_on = 'PostalCode', right_on = 'Postal Code')
df_toronto.drop("Postal Code", axis = 1, inplace = True)
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [9]:
df_toronto.shape

(103, 5)

<h3 id="assignment_2_3">2.3: Explore and cluster the neighborhoods in Toronto</h3>

Define an instance of the geocoder for Toronto.

In [10]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Create a map of Toronto with neighborhoods superimposed on top.

In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add neighborhoods markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Narrow down to neighborhoods in **Downtown Toronto** only.

In [12]:
df_downtown = df_toronto[df_toronto['Borough'] == 'Downtown Toronto'].reset_index(drop = True)
df_downtown.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [13]:
df_downtown.shape

(17, 5)

Create a map of **Downtown Toronto** with neighborhoods superimposed on top.

In [14]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude, longitude], zoom_start=12.5)

# add neighborhoods markers to map
for lat, lng, borough, neighborhood in zip(df_downtown['Latitude'], df_downtown['Longitude'], df_downtown['Borough'], df_downtown['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

Define Foursquare credentials and version.

In [15]:
CLIENT_ID = 'RIYT5OPWMC205ZAGHGCHANWVHPBEQ2HJSTIRKLYXIAY0AWOJ'
CLIENT_SECRET = 'CTFKGNQ13Y5ISMAXCEZL1ZB40WHZB1IXG0E44ZFBRLSXKCMD'
VERSION = '20180605'
LIMIT = 100

Explore the first neighborhood in **Downtown Toronto**.

In [16]:
# get the neighborhood name
df_downtown.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

In [17]:
# get the neighborhood's latitude and longitude values
neighborhood_latitude = df_downtown.loc[0, 'Latitude']
neighborhood_longitude = df_downtown.loc[0, 'Longitude']

neighborhood_name = df_downtown.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [47]:
# get the top 50  venues that are in Regent Park, Harbourfront within a radius of 500 meters
LIMIT = 50
radius = 500

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# get the results in a json format
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60f322974b10aa357b0d91fe'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 46,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [48]:
# clean the json and structure it into a pandas dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(100)

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980
5,Corktown Common,Park,43.655618,-79.356211
6,Dominion Pub and Kitchen,Pub,43.656919,-79.358967
7,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
8,The Extension Room,Gym / Fitness Center,43.653313,-79.359725
9,The Distillery Historic District,Historic Site,43.650244,-79.359323


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

46 venues were returned by Foursquare.


Explore neighborhood in **Downtown Toronto**.

In [22]:
# function to repeat the same process to all neighborhoods in Downtown Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
downtown_toronto_venues = getNearbyVenues(names=df_downtown['Neighborhood'],
                                   latitudes=df_downtown['Latitude'],
                                   longitudes=df_downtown['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [24]:
downtown_toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [25]:
downtown_toronto_venues.shape

(715, 7)

In [26]:
# check how many venues were returned for each neighborhood
downtown_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,50,50,50,50,50,50
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",18,18,18,18,18,18
Central Bay Street,50,50,50,50,50,50
Christie,16,16,16,16,16,16
Church and Wellesley,50,50,50,50,50,50
"Commerce Court, Victoria Hotel",50,50,50,50,50,50
"First Canadian Place, Underground city",50,50,50,50,50,50
"Garden District, Ryerson",50,50,50,50,50,50
"Harbourfront East, Union Station, Toronto Islands",50,50,50,50,50,50


In [27]:
# check how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 180 uniques categories.


Analyze each neighborhood in **Downtown Toronto**.

In [43]:
# one hot encoding
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_toronto_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_toronto_onehot.columns[-1]] + list(downtown_toronto_onehot.columns[:-1])
downtown_toronto_onehot = downtown_toronto_onehot[fixed_columns]

downtown_toronto_onehot.head(100)

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
# group rows by neighborhood and y taking the mmean of the frequency of occurence of each category
downtown_toronto_grouped = downtown_toronto_onehot.groupby('Neighborhood').mean().reset_index()
downtown_toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00,0.020000,0.000000,0.00,0.00,0.00,0.02,0.000000,0.00,0.00
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.00,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.00,...,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00
2,Central Bay Street,0.020000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.02,0.000000,0.000000,0.00,0.00,0.00,0.02,0.000000,0.00,0.00
3,Christie,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00
4,Church and Wellesley,0.040000,0.02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00,0.020000,0.000000,0.02,0.00,0.00,0.00,0.000000,0.00,0.00
5,"Commerce Court, Victoria Hotel",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,...,0.00,0.000000,0.000000,0.00,0.00,0.00,0.02,0.000000,0.00,0.00
6,"First Canadian Place, Underground city",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,...,0.02,0.000000,0.000000,0.00,0.00,0.00,0.02,0.000000,0.00,0.00
7,"Garden District, Ryerson",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.02,0.020000,0.040000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00,0.000000,0.020000,0.00,0.00,0.02,0.00,0.000000,0.00,0.00
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.02,0.000000,0.000000,0.00,0.00,0.00,0.06,0.000000,0.04,0.02


In [30]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in downtown_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_toronto_grouped[downtown_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.06
1      Beer Bar  0.04
2      Pharmacy  0.04
3        Bakery  0.04
4  Cocktail Bar  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0      Airport Service  0.17
1       Airport Lounge  0.11
2     Airport Terminal  0.11
3  Rental Car Location  0.06
4          Coffee Shop  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1                Café  0.06
2      Sandwich Place  0.06
3  Italian Restaurant  0.04
4     Bubble Tea Shop  0.04


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3    Coffee Shop  0.06
4     Baby Store  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.06
1          Yoga Studio  0.04
2     Sushi Restaurant  0.04
3  Japanese Restaurant  0.04
4           Restaurant  0.0

In [31]:
# function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
# create a dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Beer Bar,Pharmacy,Bakery,Cocktail Bar,Seafood Restaurant,Farmers Market,Restaurant,Cheese Shop,Moroccan Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Rental Car Location,Coffee Shop,Sculpture Garden,Boutique,Plane,Harbor / Marina,Bar
2,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Burger Joint,Ice Cream Shop,Dessert Shop,Salad Place,Gym / Fitness Center
3,Christie,Grocery Store,Café,Park,Coffee Shop,Baby Store,Athletics & Sports,Restaurant,Italian Restaurant,Candy Store,Nightclub
4,Church and Wellesley,Coffee Shop,Yoga Studio,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Men's Store,Creperie,Pizza Place,Hobby Shop


Cluster neighboorhoods in **Downtown Toronto**.

Run _k_-means to cluster the neighborhood into 5 clusters.

In [33]:
# set number of clusters
kclusters = 5

downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 2, 0, 3, 4, 0, 0, 4, 0, 4], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_toronto_merged = df_downtown

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
downtown_toronto_merged = downtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Café,Theater,Breakfast Spot,Yoga Studio,Event Space,Beer Store
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,Coffee Shop,Café,Theater,Cosmetics Shop,Clothing Store,Ramen Restaurant,Fast Food Restaurant,Bookstore,Japanese Restaurant,Furniture / Home Store
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Café,Cosmetics Shop,Coffee Shop,Creperie,Farmers Market,Bakery,Japanese Restaurant,Gastropub,Seafood Restaurant,Park
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,4,Coffee Shop,Beer Bar,Pharmacy,Bakery,Cocktail Bar,Seafood Restaurant,Farmers Market,Restaurant,Cheese Shop,Moroccan Restaurant
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Burger Joint,Ice Cream Shop,Dessert Shop,Salad Place,Gym / Fitness Center


Finally, let's visualize the resulting clusters.

In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12.5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighborhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters.

#### Cluster 1

In [36]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Park,Bakery,Pub,Café,Theater,Breakfast Spot,Yoga Studio,Event Space,Beer Store
4,Downtown Toronto,0,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Burger Joint,Ice Cream Shop,Dessert Shop,Salad Place,Gym / Fitness Center
6,Downtown Toronto,0,Coffee Shop,Café,Restaurant,American Restaurant,Thai Restaurant,Pizza Place,Steakhouse,Concert Hall,Seafood Restaurant,Deli / Bodega
7,Downtown Toronto,0,Coffee Shop,Aquarium,Café,Plaza,Park,Brewery,Hotel,Juice Bar,Lake,Bubble Tea Shop
8,Downtown Toronto,0,Coffee Shop,Café,Hotel,Seafood Restaurant,Restaurant,Japanese Restaurant,Bakery,Pub,Museum,Plaza
9,Downtown Toronto,0,Café,Coffee Shop,Hotel,Restaurant,Seafood Restaurant,Gym,Japanese Restaurant,Gastropub,American Restaurant,Bakery
15,Downtown Toronto,0,Café,Coffee Shop,Restaurant,Seafood Restaurant,Gym,Hotel,Japanese Restaurant,American Restaurant,Deli / Bodega,Concert Hall


#### Cluster 2

In [37]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Museum,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop


#### Cluster 3

In [38]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,2,Airport Service,Airport Lounge,Airport Terminal,Rental Car Location,Coffee Shop,Sculpture Garden,Boutique,Plane,Harbor / Marina,Bar


#### Cluster 4

In [39]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,3,Grocery Store,Café,Park,Coffee Shop,Baby Store,Athletics & Sports,Restaurant,Italian Restaurant,Candy Store,Nightclub


#### Cluster 5

In [40]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 4, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,4,Coffee Shop,Café,Theater,Cosmetics Shop,Clothing Store,Ramen Restaurant,Fast Food Restaurant,Bookstore,Japanese Restaurant,Furniture / Home Store
2,Downtown Toronto,4,Café,Cosmetics Shop,Coffee Shop,Creperie,Farmers Market,Bakery,Japanese Restaurant,Gastropub,Seafood Restaurant,Park
3,Downtown Toronto,4,Coffee Shop,Beer Bar,Pharmacy,Bakery,Cocktail Bar,Seafood Restaurant,Farmers Market,Restaurant,Cheese Shop,Moroccan Restaurant
10,Downtown Toronto,4,Café,Bakery,Japanese Restaurant,Bookstore,Sandwich Place,Bar,Yoga Studio,Poutine Place,Beer Store,Restaurant
11,Downtown Toronto,4,Café,Vegetarian / Vegan Restaurant,Bar,Vietnamese Restaurant,Mexican Restaurant,Bakery,Coffee Shop,Burger Joint,Caribbean Restaurant,Wine Bar
14,Downtown Toronto,4,Coffee Shop,Italian Restaurant,Bakery,Restaurant,Café,Chinese Restaurant,Pub,Pizza Place,Pharmacy,Pet Store
16,Downtown Toronto,4,Coffee Shop,Yoga Studio,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Men's Store,Creperie,Pizza Place,Hobby Shop
